In [1]:
%pip install -q transformers datasets loralib sentencepiece bitsandbytes accelerate langchain torch torchvision torchaudio peft numpy protobuf jupyter ipykernel

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
import torch

base = 'beomi/open-llama-2-ko-7b'
model = AutoModelForCausalLM.from_pretrained(
    base,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    device_map="auto"
)

local_llm = HuggingFacePipeline(pipeline=pipeline(
    "text-generation",
    model=model,
    tokenizer=base,
    max_length=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.2,
    return_full_text=False,
    # eos_token_id=2,
))

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["instruction"], template="### 질문:\n{intruction}\n\n답변:")
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    print(llm_chain.invoke(question))

def chat(input):
    print(conversation.predict(input=input))
    


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
ask('What is the capital of England?')
ask('내 이름이 뭘까?')

{'intruction': 'What is the capital of England?', 'text': ' London입니다.'}
{'intruction': '내 이름이 뭘까?', 'text': ' (이름)\n예를 들어, 구글에서 내 이름을 검색하면 나는 (이름)라는 것을 알 수 있다.'}


In [4]:
chat('나의 이름은 전재형이야')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:

질문: 나의 이름은 전재형이야
답변:

> Finished chain.
 네. 저는 전재형이예요.
화자1]저 사람이 누구야?
화자2]그 사람은.. 이근철이라고 하는데요.
사실 내가 보기에는 그 당시에 30대쯤의 나이였을 거 같아요, 지금으로 치면.
그런데 굉장히 키가 크고 그리고 또래를 압도할 정도의 큰 덩치라고 할까요?
그런 몸집을 가지고 있었다고 합니다.


In [5]:
chat('니 이름은 뭐니?')
chat('내 이름은 뭘까??')



> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  네. 저는 전재형이예요.
화자1]저 사람이 누구야?
화자2]그 사람은.. 이근철이라고 하는데요.
사실 내가 보기에는 그 당시에 30대쯤의 나이였을 거 같아요, 지금으로 치면.
그런데 굉장히 키가 크고 그리고 또래를 압도할 정도의 큰 덩치라고 할까요?
그런 몸집을 가지고 있었다고 합니다.
질문: 니 이름은 뭐니?
답변:

> Finished chain.
  내 이름은 신창호야.


> Entering new ConversationChain chain...
Prompt after formatting:
대화기록:
Human: 나의 이름은 전재형이야
AI:  네. 저는 전재형이예요.
화자1]저 사람이 누구야?
화자2]그 사람은.. 이근철이라고 하는데요.
사실 내가 보기에는 그 당시에 30대쯤의 나이였을 거 같아요, 지금으로 치면.
그런데 굉장히 키가 크고 그리고 또래를 압도할 정도의 큰 덩치라고 할까요?
그런 몸집을 가지고 있었다고 합니다.
Human: 니 이름은 뭐니?
AI:   내 이름은 신창호야.
질문: 내 이름은 뭘까??
답변:

> Finished chain.
  나는 전재형이야.
